In [1]:
import gymnasium
from gymnasium import spaces
import numpy as np
import pandas as pd
import heapq
import matplotlib.pyplot as plt
from collections import defaultdict
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
import os

class RadarTaskSchedulerEnv(gymnasium.Env):
    def __init__(self, task_file="dataset2.csv", render_mode=None):
        super(RadarTaskSchedulerEnv, self).__init__()

        self.tasks_df = pd.read_csv(task_file)
        self.num_tasks = len(self.tasks_df)

        self.priority_queue = []
        self.entry_count = 0

        self.current_task_data = None

        self.scheduled_tasks_log = []

        self.observation_space = spaces.Box(low=0, high=100, shape=(3,), dtype=np.float32)

        self.action_space = spaces.Discrete(3 * 3 * 3)

        self.render_mode = render_mode
        self._last_info = {}
        self._last_reward = 0

    def reset(self,*, seed=None, options=None):
        super().reset(seed=seed)

        self.priority_queue = []
        self.entry_count = 0

        for index, row in self.tasks_df.iterrows():
            negative_priority = -row['Priority']
            heapq.heappush(self.priority_queue, (negative_priority, self.entry_count, row.to_dict()))
            self.entry_count += 1
        
        self.scheduled_tasks_log = [] 
        self._last_info = {}
        self._last_reward = 0

        obs = self._get_observation()
        
        return obs, {}

    def step(self, action_index):
        if not self.priority_queue:
            return np.array([0, 0, 0], dtype=np.float32), 0, True, False, {
                "task_dropped": False, 
                "task_id": "N/A", 
                "message": "No tasks left to process.",
                "dropped_tasks_count": 0
            }

        _priority, _entry_count, task_to_process = heapq.heappop(self.priority_queue)
        
        delay = action_index // 9
        compress = (action_index % 9) // 3
        radar_choice = action_index % 3
        
        tns = task_to_process['Request_Time'] + delay
        Pn = task_to_process['Init_Power'] + compress

        task_dropped = (tns > task_to_process['Deadline']) or \
                       (Pn > task_to_process['Max_Power'])

        reward = 0.0

        if not task_dropped:
            reward += 1.0
        else:
            reward -= 1.0

        if not task_dropped:
            time_to_deadline = task_to_process['Deadline'] - tns
            if time_to_deadline < 2:
                reward -= 0.1
            elif time_to_deadline < 0:
                reward -= 0.2
        elif tns > task_to_process['Deadline'] + 5:
             reward -= 0.2

        if not task_dropped:
            power_margin = task_to_process['Max_Power'] - Pn
            if power_margin < 2:
                reward -= 0.1
            elif power_margin < 0:
                reward -= 0.2
        elif Pn > task_to_process['Max_Power'] + 5:
            reward -= 0.2

        if delay == 0 and not task_dropped:
            reward += 0.05

        if compress == 0 and not task_dropped:
            reward += 0.05

        info = { 
            "task_id": task_to_process["Task_ID"],
            "task_dropped": task_dropped,
            "delay_action": delay,
            "compress_action": compress,
            "radar_choice_action": radar_choice,
            "calculated_tns": tns,
            "calculated_Pn": Pn,
            "original_deadline": task_to_process['Deadline'],
            "original_max_power": task_to_process['Max_Power']
        }
        self._last_info = info
        self._last_reward = reward

        self.scheduled_tasks_log.append({
            'Task_ID': task_to_process['Task_ID'],
            'Priority': task_to_process['Priority'],
            'Action_Delay': delay,
            'Action_Compress': compress,
            'Action_Radar': radar_choice,
            'Dropped': task_dropped,
            'Final_TNS': tns,
            'Final_Pn': Pn,
            'Original_Deadline': task_to_process['Deadline'],
            'Original_Max_Power': task_to_process['Max_Power']
        })

        done = not self.priority_queue

        if done:
            info['dropped_tasks_count'] = sum(1 for task in self.scheduled_tasks_log if task['Dropped'])
        else:
            info['dropped_tasks_count'] = 0

        next_obs = self._get_observation()
            
        return next_obs, reward, done, False, info

    def _get_observation(self):
        if self.priority_queue:
            _priority, _entry_count, task_dict = self.priority_queue[0] 
            self.current_task_data = task_dict
            return np.array([task_dict['Duration'], task_dict['Deadline'], task_dict['Init_Power']], dtype=np.float32)
        else:
            self.current_task_data = None
            return np.array([0, 0, 0], dtype=np.float32)

    def close(self):
        pass

class EpisodeLoggerCallback(BaseCallback):
    def __init__(self, output_file, tasks_per_episode, total_episodes_target, verbose=0):
        super(EpisodeLoggerCallback, self).__init__(verbose)
        self.output_file = output_file
        self.tasks_per_episode = tasks_per_episode
        self.total_episodes_target = total_episodes_target 
        self.episode_num = 0
        self.reward_log = []
        self.task_drop_log = []

    def _on_step(self) -> bool:
        if self.locals['dones'][0]:
            self.episode_num += 1
            
            total_reward_episode = 0
            task_drops_episode = 0

            last_step_info = self.locals['infos'][0]

            if 'episode' in last_step_info:
                episode_stats = last_step_info['episode']
                total_reward_episode = episode_stats['r']
                
                task_drops_episode = last_step_info.get('dropped_tasks_count', 0)
                
                self.reward_log.append(total_reward_episode)
                self.task_drop_log.append(task_drops_episode)

                epsilon = self.model.exploration_rate if hasattr(self.model, 'exploration_rate') else 'N/A'

                log_message = f"Episode {self.episode_num}/{self.total_episodes_target} — Total Reward: {total_reward_episode:.2f} — Tasks Dropped: {task_drops_episode} — Epsilon (approx): {epsilon:.3f}"
                print(log_message)
                self.output_file.write(log_message + "\n")
        return True

    def _on_rollout_end(self) -> None:
        pass

    def _on_training_end(self) -> None:
        pass


if __name__ == "__main__":
    
    env = RadarTaskSchedulerEnv(task_file="dataset2.csv", render_mode=None)
    env = Monitor(env)

    dqn_hyperparams = {
        "learning_rate": 0.0005,
        "buffer_size": 10000,
        "learning_starts": 500,
        "batch_size": 32,
        "gamma": 0.995,
        "train_freq": (1, "step"),
        "gradient_steps": 1,
        "target_update_interval": 100,
        "exploration_fraction": 0.1,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "max_grad_norm": 10,
        "seed": 42
    }

    model = DQN("MlpPolicy", env, verbose=0, **dqn_hyperparams, device="auto")

    episodes_to_train = 1000 
    tasks_per_episode = len(pd.read_csv("dataset2.csv"))
    total_timesteps_for_training = episodes_to_train * tasks_per_episode

    output_filename = "dqn_learning_results.txt"
    
    with open(output_filename, "w") as f_out:
        f_out.write("--- DQN Training Log ---\n\n")
        print("Starting DQN training...")
        f_out.write("Starting DQN training...\n\n")

        episode_logger_callback = EpisodeLoggerCallback(
            output_file=f_out, 
            tasks_per_episode=tasks_per_episode,
            total_episodes_target=episodes_to_train
        )

        model.learn(total_timesteps=total_timesteps_for_training, callback=episode_logger_callback)

        print("\nTraining complete!")
        f_out.write("\nTraining complete!\n")

        model_path = "dqn_radar_scheduler_final_model"
        model.save(model_path)
        print(f"Final model saved to {model_path}")
        f_out.write(f"Final model saved to {model_path}\n")

        reward_log = episode_logger_callback.reward_log
        task_drop_log = episode_logger_callback.task_drop_log

        window_size = 50

        if not reward_log:
            print("\nReward log is empty. No reward plot will be generated.")
            f_out.write("\nReward log is empty. No reward plot will be generated.\n")
        else:
            moving_avg = []
            x_moving_avg = []
            if len(reward_log) >= window_size:
                moving_avg = np.convolve(reward_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg = range(window_size - 1, len(reward_log))

            plt.figure(figsize=(14,7))
            plt.plot(reward_log, label='Episode Rewards', color='orange', alpha=0.7, linewidth=1)
            
            if moving_avg.size > 0:
                plt.plot(x_moving_avg, moving_avg, label=f'Moving Average ({window_size} episodes)', color='blue', linewidth=2)
            
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Total Reward', fontsize=12)
            plt.title('Total Reward over Episodes (DQN)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_reward_plot.png")
            plt.close()
            print(f"Reward plot saved as dqn_reward_plot.png")
            f_out.write(f"Reward plot saved as dqn_reward_plot.png\n")

        if not task_drop_log:
            print("\nTask drop log is empty. No task drop plot will be generated.")
            f_out.write("\nTask drop log is empty. No task drop plot will be generated.\n")
        else:
            moving_avg_drops = []
            x_moving_avg_drops = []
            if len(task_drop_log) >= window_size:
                moving_avg_drops = np.convolve(task_drop_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_drops = range(window_size - 1, len(task_drop_log))

            plt.figure(figsize=(14,7))
            plt.plot(task_drop_log, label='Tasks Dropped per Episode', color='red', alpha=0.8, linewidth=1)
            
            if moving_avg_drops.size > 0:
                plt.plot(x_moving_avg_drops, moving_avg_drops, label=f'Moving Average ({window_size} episodes)', color='darkred', linewidth=2)
            
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Dropped', fontsize=12)
            plt.title('Number of Tasks Dropped per Episode (DQN)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_task_drop_plot.png")
            plt.close()
            print(f"Task drop plot saved as dqn_task_drop_plot.png")
            f_out.write(f"Task drop plot saved as dqn_task_drop_plot.png\n")

        f_out.write("\n\n--- Running Final Evaluation Episode for Schedule and Metrics ---\n")
        print("\n--- Running Final Evaluation Episode for Schedule and Metrics ---")

        eval_env = RadarTaskSchedulerEnv(task_file="dataset2.csv", render_mode=None) 

        obs, _ = eval_env.reset()
        done = False
        
        eval_env.scheduled_tasks_log = []

        while not done:
            action_index, _states = model.predict(obs, deterministic=True)
            obs, reward, done, _, info = eval_env.step(action_index)

        f_out.write("\n--- Final Task Schedule (from last evaluation) ---\n")
        print("\n--- Final Task Schedule (from last evaluation) ---")
        if eval_env.scheduled_tasks_log and len(eval_env.scheduled_tasks_log) == eval_env.num_tasks:
            schedule_df = pd.DataFrame(eval_env.scheduled_tasks_log)
            f_out.write(schedule_df.to_string(index=False))
            print(schedule_df.to_string(index=False))
        else:
            f_out.write("No full schedule generated in the final evaluation. This might mean the environment did not complete a full episode (e.g., due to an empty task queue).\n")
            print("No full schedule generated in the final evaluation. This might mean the environment did not complete a full episode (e.g., due to an empty task queue).")

        f_out.write("\n\n--- Performance Metrics (from last evaluation) ---\n")
        print("\n--- Performance Metrics (from last evaluation) ---")

        if eval_env.scheduled_tasks_log:
            total_tasks_attempted = len(eval_env.scheduled_tasks_log)
            completed_tasks = sum(1 for task in eval_env.scheduled_tasks_log if not task['Dropped'])
            dropped_tasks_count = sum(1 for task in eval_env.scheduled_tasks_log if task['Dropped'])

            if total_tasks_attempted > 0:
                percent_tasks_completed = (completed_tasks / total_tasks_attempted) * 100
                percent_tasks_dropped = (dropped_tasks_count / total_tasks_attempted) * 100
            else:
                percent_tasks_completed = 0.0
                percent_tasks_dropped = 0.0

            metrics = {
                "Total Tasks in Dataset": eval_env.num_tasks,
                "Tasks Attempted (in last evaluation)": total_tasks_attempted,
                "Tasks Successfully Completed": completed_tasks,
                "Tasks Dropped": dropped_tasks_count,
                "Percentage of Tasks Completed": f"{percent_tasks_completed:.2f}%",
                "Percentage of Tasks Dropped": f"{percent_tasks_dropped:.2f}%"
            }

            metrics_df = pd.DataFrame(metrics.items(), columns=['Metric', 'Value'])
            f_out.write(metrics_df.to_string(index=False))
            print(metrics_df.to_string(index=False))
        else:
            f_out.write("No metrics could be calculated from the final evaluation as no tasks were processed.\n")
            print("No metrics could be calculated from the final evaluation as no tasks were processed.")

    env.close()
    eval_env.close()
    print("All environments closed.")

Starting DQN training...
Episode 1/1000 — Total Reward: 98.35 — Tasks Dropped: 2 — Epsilon (approx): 0.991
Episode 2/1000 — Total Reward: 98.35 — Tasks Dropped: 2 — Epsilon (approx): 0.981
Episode 3/1000 — Total Reward: 94.50 — Tasks Dropped: 4 — Epsilon (approx): 0.972
Episode 4/1000 — Total Reward: 98.75 — Tasks Dropped: 2 — Epsilon (approx): 0.962
Episode 5/1000 — Total Reward: 96.40 — Tasks Dropped: 3 — Epsilon (approx): 0.953
Episode 6/1000 — Total Reward: 98.45 — Tasks Dropped: 2 — Epsilon (approx): 0.943
Episode 7/1000 — Total Reward: 97.80 — Tasks Dropped: 2 — Epsilon (approx): 0.934
Episode 8/1000 — Total Reward: 99.70 — Tasks Dropped: 1 — Epsilon (approx): 0.924
Episode 9/1000 — Total Reward: 96.90 — Tasks Dropped: 3 — Epsilon (approx): 0.915
Episode 10/1000 — Total Reward: 98.25 — Tasks Dropped: 2 — Epsilon (approx): 0.905
Episode 11/1000 — Total Reward: 98.35 — Tasks Dropped: 2 — Epsilon (approx): 0.896
Episode 12/1000 — Total Reward: 98.25 — Tasks Dropped: 2 — Epsilon (app